In [ ]:
#!pip install textblob

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re
import json
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

#Metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

In [2]:
#import library
!pip install contractions 

import pandas as pd
import numpy as np

import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import unicodedata
import contractions
from contractions import contractions_dict
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re


import re, string, unicodedata
import nltk
import contractions
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 8.1 MB/s eta 0:00:00


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
import pandas as pd

# Read JSONL file into a DataFrame
df = pd.read_json('/content/data.jsonl', lines=True)

# Print the first few rows of the DataFrame
print(df.head())


ValueError: ignored

In [13]:
import pandas as pd
import json

# Open the JSONL file
with open('/content/data.jsonl', 'r') as f:
    # Read each line and load it as a JSON object
    lines = [json.loads(line) for line in f.readlines()]

# Convert the list of JSON objects into a DataFrame
df = pd.DataFrame(lines)

# Print the first few rows of the DataFrame
print(df.head())

                                                text  label
0  i feel awful about it too because it s my job ...      0
1                              im alone i feel awful      0
2  ive probably mentioned this before but i reall...      1
3           i was feeling a little low few days back      0
4  i beleive that i am much more sensitive to oth...      2


In [14]:
df = df[df['label'].isin([2, 5]) == False]

In [15]:
df.replace({4: 2}, inplace=True)

In [16]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text



# Define function to expand contractions
def expand_contractions(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


# special_characters removal
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


def remove_punctuation_and_splchars(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_word = remove_special_characters(new_word, True)
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

stopword_list= stopwords.words('english')
stopword_list.append('im')
stopword_list.append('ive')

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopword_list:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation_and_splchars(words)
    words = remove_stopwords(words)
    return words

def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

def normalize_and_lemmaize(input):
    sample = denoise_text(input)
    sample = remove_special_characters(sample)
    words = nltk.word_tokenize(sample)
    words = normalize(words)
    lemmas = lemmatize(words)
    return ' '.join(lemmas)

In [17]:
print(stopword_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [18]:
df['clean_data'] = df['text'].map(lambda text: normalize_and_lemmaize(text))

In [19]:
labels = df['label'].unique()
if len(labels) < 2:
    print('Error: Only one label in the data')
else:
    X = df[['clean_data']]
    y = df[['label']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                        random_state=0,
                                                        shuffle=True,
                                                        stratify=y)


### TF-IDF + ML

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression

# Define the pipeline steps
tfidf_vectorizer = TfidfVectorizer()
logreg = LogisticRegression(max_iter=1000000)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 1))),
    ('scale', MaxAbsScaler()),
    ('logreg', logreg)
])

# Train the model using the pipeline
pipeline.fit(X_train['clean_data'], y_train['label'])

# Make predictions on test data
y_pred = pipeline.predict(X_test['clean_data'])

In [ ]:
# Accuracy score
accuracy = pipeline.score(X_test['clean_data'] , y_test)
# Assign f1 score to a variable
score = f1_score(y_test, y_pred, average = 'weighted')

# Printing evaluation metric (f1-score) 
print("Accuracy: {}".format(accuracy))
print("f1 score: {}".format(score))
print(classification_report(y_test,y_pred))

Accuracy: 0.9301768381502103
f1 score: 0.9300919818124156
              precision    recall  f1-score   support

           0       0.93      0.93      0.93     24238
           1       0.96      0.97      0.96     28214
           2       0.88      0.88      0.88      9542
           3       0.89      0.89      0.89     11463

    accuracy                           0.93     73457
   macro avg       0.92      0.91      0.92     73457
weighted avg       0.93      0.93      0.93     73457



In [ ]:
# Define a new text sample to predict
new_text = "i was feeling a little low few days back."

# Make predictions using the trained pipeline
prediction = pipeline.predict([new_text])

# Print the predicted label
print("Predicted label:", prediction[0])
print('0: Sadness, 1: Joy, 2: Fear, 3: Anger')

Predicted label: 0
0: Sadness, 1: Joy, 2: Fear, 3: Anger


### TFIDF + LinearSVC

In [ ]:
labels = df['label'].unique()
if len(labels) < 2:
    print('Error: Only one label in the data')
else:
    X = df[['clean_data']]
    y = df[['label']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                        random_state=0,
                                                        shuffle=True,
                                                        stratify=y)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.svm import LinearSVC

# Define the pipeline steps
tfidf_vectorizer = TfidfVectorizer()
linear_svc = LinearSVC()

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 1))),
    ('scale', MaxAbsScaler()),
    ('linear_svc', linear_svc)
])

# Train the model using the pipeline
pipeline.fit(X_train['clean_data'], y_train['label'])

# Make predictions on test data
y_pred = pipeline.predict(X_test['clean_data'])


In [ ]:
# Accuracy score
accuracy = pipeline.score(X_test['clean_data'] , y_test)
# Assign f1 score to a variable
score = f1_score(y_test, y_pred, average = 'weighted')

# Printing evaluation metric (f1-score) 
print("Accuracy: {}".format(accuracy))
print("f1 score: {}".format(score))
print(classification_report(y_test,y_pred))

Accuracy: 0.9283662550880106
f1 score: 0.9283107570748977
              precision    recall  f1-score   support

           0       0.93      0.93      0.93     24238
           1       0.96      0.97      0.96     28214
           3       0.89      0.88      0.88     11463
           4       0.87      0.88      0.88      9542

    accuracy                           0.93     73457
   macro avg       0.91      0.91      0.91     73457
weighted avg       0.93      0.93      0.93     73457



In [ ]:
# Define a new text sample to predict
new_text = "i was feeling a little low few days back"

# Make predictions using the trained pipeline
prediction = pipeline.predict([new_text])

# Print the predicted label
print("Predicted label:", prediction[0])
print('0: Sadness, 1: Joy, 3: Anger, 4: Fear')

Predicted label: 0
0: Sadness, 1: Joy, 3: Anger, 4: Fear


### XGBoost

In [ ]:
labels = df['label'].unique()
if len(labels) < 2:
    print('Error: Only one label in the data')
else:
    X = df[['clean_data']]
    y = df[['label']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                        random_state=424,
                                                        shuffle=True,
                                                        stratify=y)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier

# Define the pipeline steps
tfidf_vectorizer = TfidfVectorizer()
xgb_classifier = XGBClassifier()

pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('xgb', xgb_classifier)
])

# Train the model using the pipeline
pipeline.fit(X_train['clean_data'], y_train['label'])

# Make predictions on test data
y_pred = pipeline.predict(X_test['clean_data'])


In [ ]:
# Accuracy score
accuracy = pipeline.score(X_test['clean_data'] , y_test)
# Assign f1 score to a variable
score = f1_score(y_test, y_pred, average = 'weighted')

# Printing evaluation metric (f1-score) 
print("Accuracy: {}".format(accuracy))
print("f1 score: {}".format(score))
print(classification_report(y_test,y_pred))

Accuracy: 0.9271955021304981
f1 score: 0.9270023330199214
              precision    recall  f1-score   support

           0       0.93      0.92      0.93     24238
           1       0.94      0.96      0.95     28214
           2       0.87      0.91      0.89      9542
           3       0.92      0.86      0.89     11463

    accuracy                           0.93     73457
   macro avg       0.92      0.91      0.92     73457
weighted avg       0.93      0.93      0.93     73457



In [ ]:
# Define a new text sample to predict
new_text = "i was feeling a little low few days back"

# Make predictions using the trained pipeline
prediction = pipeline.predict([new_text])

# Print the predicted label
print("Predicted label:", prediction[0])
print('0: Sadness, 1: Joy, 3: Anger, 4: Fear')

Predicted label: 0
0: Sadness, 1: Joy, 3: Anger, 4: Fear


### LTSM

In [20]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Read in the cleaned text data and labels
X = df['clean_data']
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the tokenizer with a maximum vocabulary size of 5000 words
tokenizer = Tokenizer(num_words=5000)

# Fit the tokenizer on the training data
tokenizer.fit_on_texts(X_train)

# Convert the text data to numerical sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to a maximum length of 100
max_seq_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_seq_length)


In [30]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert the text to sequences of integers
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to a fixed length
max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Convert the label data to a matrix using LabelBinarizer
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_len))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=5, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print('Test accuracy:', accuracy)


Epoch 1/5
9183/9183 [==============================] - 126s 13ms/step - loss: 0.1004 - accuracy: 0.9544 - val_loss: 0.0580 - val_accuracy: 0.9711
Epoch 2/5
9183/9183 [==============================] - 95s 10ms/step - loss: 0.0509 - accuracy: 0.9723 - val_loss: 0.0520 - val_accuracy: 0.9721
Epoch 3/5
9183/9183 [==============================] - 93s 10ms/step - loss: 0.0435 - accuracy: 0.9750 - val_loss: 0.0560 - val_accuracy: 0.9721
Epoch 4/5
9183/9183 [==============================] - 95s 10ms/step - loss: 0.0384 - accuracy: 0.9768 - val_loss: 0.0563 - val_accuracy: 0.9721
Epoch 5/5
2296/2296 [==============================] - 11s 5ms/step - loss: 0.0633 - accuracy: 0.9714
Test accuracy: 0.9713574051856995


In [39]:
# Define the custom text to test
custom_text = "This is creeping me out"

# Tokenize the custom text
custom_text_seq = tokenizer.texts_to_sequences([custom_text])

# Pad the sequence to the same length as used for the training data
custom_text_pad = pad_sequences(custom_text_seq, maxlen=max_len)

# Make predictions using the trained model
predictions = model.predict(custom_text_pad)

# Convert the predictions to class labels using inverse_transform
predicted_labels = lb.inverse_transform(predictions)

print('Predicted label:', predicted_labels[0])


1/1 [==============================] - 0s 27ms/step
Predicted label: 3
